# Librerias 

In [9]:
import pandas as pd
from sqlalchemy import create_engine, text
from glob import glob
import os

# Conexion 

In [7]:
# Datos de conexión
user = "root"
password = "Levp13aa"
host = "localhost"
database = "futbol_dw"

# Crear conexión
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# Probar conexión
with engine.connect() as conn:
    result = conn.execute(text("SELECT 'Conexión exitosa a MySQL!' AS mensaje;"))
    print(result.fetchone())

('Conexión exitosa a MySQL!',)


# Bronze 

In [ ]:
# carpeta donde tienes todos los CSV de equipos españoles
folder_path = r"D:\Fut_Pred\Data_equipos\2024-2025\España"   # 🔁 cámbiala a la tuya

# busca todos los .csv (o .tsv) en esa carpeta
csv_files = glob(os.path.join(folder_path, "*.csv"))

print(f"🔍 Se encontraron {len(csv_files)} archivos CSV en {folder_path}\n")

# === 3️⃣ RECORRER Y CARGAR CADA ARCHIVO ===
for csv_path in csv_files:
    try:
        # Leer el archivo (usa tabulador como separador)
        df = pd.read_csv(csv_path, sep="\t", encoding="utf-8")

        # Agregar metadatos (liga, temporada, equipo, archivo)
        df['league'] = 'LaLiga'
        df['season'] = '2024-2025'
        df['team_name'] = os.path.splitext(os.path.basename(csv_path))[0]
        df['file_source'] = os.path.basename(csv_path)

        # Guardar en MySQL (crea la tabla si no existe)
        df.to_sql("bronze_teams", engine, if_exists="append", index=False)

        print(f"✅ Cargado correctamente: {os.path.basename(csv_path)} ({len(df)} filas)")

    except Exception as e:
        print(f"❌ Error al cargar {os.path.basename(csv_path)} → {e}")

print("\n🏁 Proceso completado: todos los equipos cargados en 'bronze_teams'")
